# Brake-Reuß Beam - DIC data

We consider digital image correlation measurements from a shaker ringdown test on a beam with a bolted lap joint [1,2]. Oscillations are initialized close to its slowest 2D spectral submanifold (SSM), which will be identified and used for model reduction.
The dataset contains the displacement field (206 points in 720 mm of beam length) of the bottom beam side measured via DIC and accelerations measured via accelerometers (ACC) in 2 locations. All data refers to a single experimental trial, acquired with different measurement devices (as described in [1]). The dataset also includes backbone curves results obtained using the Peak Finding and Fitting (PFF) method based on accelerometer signals. 
Here we construct a reduced-order model using DIC data and we validate it on the acceleration signals.


This example follows https://github.com/mattiacenedese/SSMLearn/tree/main/examples/brakereussbeam

In [6]:
from scipy.io import loadmat
from ssmlearnpy import SSMLearn
import numpy as np


In [3]:
data = loadmat('data.mat')['data_BRB']

FileNotFoundError: [Errno 2] No such file or directory: 'data.mat'

In [ ]:

TimeDIC = data[0,0].item()[0]
DisplacementDIC = data[0,0].item()[1]
TimeACC = data[0,0].item()[2]
AccelerationACC = data[0,0].item()[3]
Xmesh = data[0,0].item()[4]
Units = data[0,0].item()[5]
LocationACC = data[0,0].item()[6]
PFFResultsACC = data[0,0].item()[7]

In [ ]:
time_to_plot = np.logical_and(TimeDIC<3/80, TimeDIC>0) # range to plot in the original script

In [ ]:
TT, XX = np.meshgrid(TimeDIC[time_to_plot], Xmesh)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.plot_surface(TT, XX, DisplacementDIC[:, time_to_plot[:,0]], cmap = 'inferno')
ax.set_xlabel('$t\ [s]$')
ax.set_ylabel('$x\ [mm]$')
ax.set_zlabel('$u\ [mm]$')

fig.set_tight_layout('tight')

In [ ]:
ssm = SSMLearn(
    t = [TimeDIC.ravel()], 
    x = [DisplacementDIC], 
    ssm_dim=2, 
    dynamics_type = 'flow'
)

In [7]:
from ssmlearnpy.geometry.coordinates_embedding import coordinates_embedding

In [4]:
t_y, y, opts_embedding = coordinates_embedding(ssm.emb_data['time'], ssm.emb_data['observables'],
                                               imdim = ssm.ssm_dim, over_embedding = 5)

NameError: name 'coordinates_embedding' is not defined

In [5]:
dd = loadmat('../../tests/test_BRB_from_ssmlearn.mat')['yData']

FileNotFoundError: [Errno 2] No such file or directory: '../../tests/test_BRB_from_ssmlearn.mat'

In [13]:
ssm = SSMLearn(
    t = t_y, 
    x = y, 
    derive_embdedding=False,
    ssm_dim=2, 
    dynamics_type = 'flow',
)

In [14]:
ssm.get_reduced_coordinates('linearchart')


In [15]:
from ssmlearnpy.utils.plots import Plot

plotter = Plot(
    label_observables = 'x',
    label_reduced_coordinates = 'eta'
)

In [1]:
fig = plotter.make_plot(ssm, data_name = 'reduced_coordinates', idx_coordinates=[1, 2])
fig.show()

NameError: name 'plotter' is not defined